# Problem 1

## Part 1.Explain the importance of interpretability when training machine learning models. Why is model explainability necessary?

The importance of interpretability of ML model is to let every user who don't know ML easlier to understand how the model makes decisions thus they will trust the model better. It also help us to debug and identify the errors in the model to better refine the model. In this case, it can help decision-makers act confidently. Also regulations require models to explain their outputs as well. A lack of explainability can lead to legal consequences if decisions cannot be justified.

## Part 2. Perform feature selection for the Diabetes regression dataset using FeatureSelector(). How many features are discarded?

In [ ]:
pip install autofeat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.7 MB/s eta 0:00:00


In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
from autofeat import FeatureSelector

In [ ]:
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

feature_selector = FeatureSelector()
X_selected = feature_selector.fit_transform(X, y)
num_original_features = X.shape[1]
num_selected_features = X_selected.shape[1]
features_discarded = num_original_features - num_selected_features

print(f"Original features: {num_original_features}")
print(f"Selected features: {num_selected_features}")
print(f"Features discarded: {features_discarded}")

Original features: 10
Selected features: 6
Features discarded: 4


/usr/local/lib/python3.10/dist-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:


## Part 3. Perform a train-test split on your dataset. Select a regression model from skLearn and fit it to the training dataset. What is the R2 score on the training and test set?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

r2_score_train = r2_score(y_train, y_train_pred)
r2_score_test = r2_score(y_test, y_test_pred)

print(f"R2 score on training set: {r2_score_train}")
print(f"R2 score on test set: {r2_score_test}")

R2 score on training set: 0.5212927960835636
R2 score on test set: 0.46323772750271597


## Part 4. Keepingthetrainandtestdatasetthesame,run3featureengineeringstepsusingAutoFeatRegressor(). What is the R2 score on the training and test set now? Mention any five new features generated by the output of AutoFeatRegressor().

In [ ]:
from autofeat import AutoFeatRegressor

auto_feat = AutoFeatRegressor(verbose=1, feateng_steps=3)

X_train_auto = auto_feat.fit_transform(X_train, y_train)

X_test_auto = auto_feat.transform(X_test)


[featsel] Scaling data...done.


/usr/local/lib/python3.10/dist-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:


In [ ]:

model_auto = LinearRegression()
model_auto.fit(X_train_auto, y_train)

y_train_pred_auto = model_auto.predict(X_train_auto)
y_test_pred_auto = model_auto.predict(X_test_auto)

r2_train_auto = r2_score(y_train, y_train_pred_auto)
r2_test_auto = r2_score(y_test, y_test_pred_auto)

print(f"R^2 score on training set with AutoFeat: {r2_train_auto:.4f}")
print(f"R^2 score on test set with AutoFeat: {r2_test_auto:.4f}")


R^2 score on training set with AutoFeat: 0.6013
R^2 score on test set with AutoFeat: 0.4603


In [ ]:
# Get the names of the generated features
new_features = set(X_train_auto.columns) - set(X_train.columns)
print(f"New features generated by AutoFeat: {list(new_features)[:5]}")


New features generated by AutoFeat: ['s3/Abs(bmi)', '1/(1/bp + 1/bmi)', '1/(bmi - s1**2)', 'exp(bmi)*exp(bp)', 'exp(s1**3/bp)']


# Problem 2.


## Part1.

In [ ]:
!pip install -U "ray[tune]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 29.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

import ray
from ray import tune
from ray.train.tensorflow.keras import ReportCheckpointCallback

from ray.train.tensorflow.keras import ReportCheckpointCallback

In [ ]:
from ray import tune
from ray.train.tensorflow.keras import ReportCheckpointCallback
import tensorflow as tf
from ray.air import session

def train_mnist(config):
    batch_size = config["batch_size"]
    epochs = 12

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(5, 5), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(filters=2 * config["conv_filters"], kernel_size=(5, 5), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation="relu"),
        tf.keras.layers.Dense(84, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    history = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        verbose=0
    )

    # Report metrics
    session.report({
        "mean_accuracy": history.history["accuracy"][-1],
        # "val_accuracy": history.history["val_accuracy"][-1]
    })


In [ ]:
import ray

# 清理之前的 Ray 实例
ray.shutdown()

# 重新初始化
ray.init()
print(ray.cluster_resources())


2024-12-05 15:02:21,625	INFO worker.py:1821 -- Started a local Ray instance.


{'accelerator_type:A100': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'memory': 53632175310.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 26816087654.0, 'GPU': 1.0}


In [ ]:
ray.init(ignore_reinit_error=True)
print(ray.cluster_resources())

2024-12-05 15:02:22,753	INFO worker.py:1654 -- Calling ray.init() again after it has already been called.


{'accelerator_type:A100': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'object_store_memory': 26816087654.0, 'node:172.28.0.12': 1.0, 'memory': 53632175310.0, 'GPU': 1.0}


In [ ]:
import ray
ray.shutdown()  # 清理之前的 Ray 状态
ray.init()

2024-12-05 15:02:26,568	INFO worker.py:1821 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.40.0


## Part 2.

#### Grid Search

In [ ]:
import time
start_time = time.time()
analysis = tune.run(
    train_mnist,
    resources_per_trial={"cpu": 1, "gpu": 0.5},
    config={
        "conv_filters": tune.grid_search([64, 128, 256]),
        "lr": tune.grid_search([0.001, 0.01, 0.1]),
        "dropout": tune.uniform(0, 1),
        "batch_size": tune.grid_search([64, 128, 256]),
    },
    metric="mean_accuracy",
    mode="max"
)
end_time = time.time()


+--------------------------------------------------------------------+
| Configuration for experiment     train_mnist_2024-12-05_14-26-51   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 27                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_mnist_2024-12-05_14-26-51
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-12-05_14-26-48_632070_1199/artifacts/2024-12-05_14-26-51/train_mnist_2024-12-05_14-26-51/driver_artifacts`

Trial status: 27 PENDING
Current time: 2024-12-05 14:26:55. Total running time: 0s
Logical resource usage: 2.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+-------------------------------------------------------------

(pid=3421) 2024-12-05 14:26:56.834145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3421) 2024-12-05 14:26:56.856656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3421) 2024-12-05 14:26:56.863182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=3421) 2024-12-05 14:26:58.091632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial train_mnist_f80bd_00000 started with configuration:
+--------------------------------------------------+
| Trial train_mnist_f80bd_00000 config             |
+--------------------------------------------------+
| batch_size                                    64 |
| conv_filters                                  64 |
| dropout                                  0.08001 |
| lr                                         0.001 |
+--------------------------------------------------+

(train_mnist pid=3421) Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
Trial train_mnist_f80bd_00001 started with configuration:
+--------------------------------------------------+
| Trial train_mnist_f80bd_00001 config             |
+--------------------------------------------------+
| batch_size                                   128 |
| conv_filters                                  64 |
| dropout                                  0.15776 |
| lr                     

(train_mnist pid=3421) 2024-12-05 14:27:01.101091: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_mnist pid=3421) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(train_mnist pid=3421) I0000 00:00:1733408824.377891    3635 service.cc:146] XLA service 0x7c155c017440 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
(train_mnist pid=3421) I0000 00:00:1733408824.377951    3635 service.cc:154]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
(pid=3422) 2024-12-05 14:26:56.830723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3422) 2024-12-05 14:26:56.852288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8


Trial status: 2 RUNNING | 25 PENDING
Current time: 2024-12-05 14:27:25. Total running time: 30s
Logical resource usage: 2.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+----------------------------------------------------------------------------------------+
| Trial name                status       dropout     conv_filters      lr     batch_size |
+----------------------------------------------------------------------------------------+
| train_mnist_f80bd_00000   RUNNING    0.0800099               64   0.001             64 |
| train_mnist_f80bd_00001   RUNNING    0.157755                64   0.001            128 |
| train_mnist_f80bd_00002   PENDING    0.56265                 64   0.001            256 |
| train_mnist_f80bd_00003   PENDING    0.0198027              128   0.001             64 |
| train_mnist_f80bd_00004   PENDING    0.76506                128   0.001            128 |
| train_mnist_f80bd_00005   PENDING    0.628439               128   0.001            256 |
| trai

...


Trial status: 25 TERMINATED | 2 RUNNING
Current time: 2024-12-05 14:46:57. Total running time: 20min 2s
Logical resource usage: 2.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Current best trial: f80bd_00007 with mean_accuracy=0.9981333613395691 and params={'conv_filters': 256, 'lr': 0.001, 'dropout': 0.00844249100058081, 'batch_size': 128}
+---------------------------------------------------------------------------------------------------------------------------------+
| Trial name                status         dropout     conv_filters      lr     batch_size        acc     iter     total time (s) |
+---------------------------------------------------------------------------------------------------------------------------------+
| train_mnist_f80bd_00025   RUNNING      0.408096               256   0.1              128                                        |
| train_mnist_f80bd_00026   RUNNING      0.878145               256   0.1              256                                

(train_mnist pid=26433) I0000 00:00:1733410030.474018   26532 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Trial status: 25 TERMINATED | 2 RUNNING
Current time: 2024-12-05 14:47:27. Total running time: 20min 32s
Logical resource usage: 2.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Current best trial: f80bd_00007 with mean_accuracy=0.9981333613395691 and params={'conv_filters': 256, 'lr': 0.001, 'dropout': 0.00844249100058081, 'batch_size': 128}
+---------------------------------------------------------------------------------------------------------------------------------+
| Trial name                status         dropout     conv_filters      lr     batch_size        acc     iter     total time (s) |
+---------------------------------------------------------------------------------------------------------------------------------+
| train_mnist_f80bd_00025   RUNNING      0.408096               256   0.1              128                                        |
| train_mnist_f80bd_00026   RUNNING      0.878145               256   0.1              256                                

2024-12-05 14:48:42,236	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_mnist_2024-12-05_14-26-51' in 0.0086s.



Trial train_mnist_f80bd_00026 finished iteration 1 at 2024-12-05 14:48:42. Total running time: 21min 47s
+--------------------------------------------------+
| Trial train_mnist_f80bd_00026 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         109.028 |
| time_total_s                             109.028 |
| training_iteration                             1 |
| mean_accuracy                            0.10482 |
+--------------------------------------------------+

Trial train_mnist_f80bd_00026 completed after 1 iterations at 2024-12-05 14:48:42. Total running time: 21min 47s

Trial status: 27 TERMINATED
Current time: 2024-12-05 14:48:42. Total running time: 21min 47s
Logical resource usage: 1.0/12 CPUs, 0.5/1 GPUs (0.0/1.0 accelerator_type:L4)
Current best trial: f80bd_00007 with mean_accuracy=0.9981333613395691 and params={'conv_filters': 256, 'lr': 0.001, 'dropout': 0.00

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)
best_trial = analysis.best_trial    # 最佳试验
best_accuracy = best_trial.last_result["mean_accuracy"]
print(f"Best accuracy: {best_accuracy:.4f}")
time_taken = end_time - start_time
print(f"Time taken: {time_taken:.2f} seconds")

Best hyperparameters found were:  {'conv_filters': 256, 'lr': 0.001, 'dropout': 0.00844249100058081, 'batch_size': 128}
Best accuracy: 0.9981
Time taken: 1310.90 seconds


#### hyperband


In [ ]:
from ray.tune.schedulers import HyperBandScheduler
hyperband_scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_accuracy',
    mode='max',
    max_t=10,
    reduction_factor=3)

In [ ]:
start_time = time.time()

analysis = tune.run(
        train_mnist,
        name="exp",
        resources_per_trial={
            "gpu": 1,
            "cpu": 4
        },
        config={
            "conv_filters": tune.randint(64,256),

            "batch_size": tune.grid_search([64, 128, 256]),
            "dropout": tune.uniform(0, 1),
            "lr": tune.uniform(0.001, 0.1)
        },
        scheduler=hyperband_scheduler,
        )
end_time = time.time()

+--------------------------------------------------------+
| Configuration for experiment     exp                   |
+--------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator |
| Scheduler                        HyperBandScheduler    |
| Number of trials                 3                     |
+--------------------------------------------------------+

View detailed results here: /root/ray_results/exp
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-12-05_14-26-48_632070_1199/artifacts/2024-12-05_14-49-31/exp/driver_artifacts`

Trial status: 3 PENDING
Current time: 2024-12-05 14:49:31. Total running time: 0s
Logical resource usage: 4.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+---------------------------------------------------------------------------------------------+
| Trial name                status       conv_filters     dropout           lr     batch_size |
+----------

(pid=27967) 2024-12-05 14:49:32.987114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=27967) 2024-12-05 14:49:33.009166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=27967) 2024-12-05 14:49:33.016040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=27967) 2024-12-05 14:49:34.194034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial train_mnist_228ce_00000 started with configuration:
+--------------------------------------------------+
| Trial train_mnist_228ce_00000 config             |
+--------------------------------------------------+
| batch_size                                    64 |
| conv_filters                                 167 |
| dropout                                  0.38638 |
| lr                                          0.05 |
+--------------------------------------------------+


(train_mnist pid=27967) 2024-12-05 14:49:36.004460: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_mnist pid=27967) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(train_mnist pid=27967) I0000 00:00:1733410178.584228   28083 service.cc:146] XLA service 0x7cb6e4005830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
(train_mnist pid=27967) I0000 00:00:1733410178.584276   28083 service.cc:154]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
(train_mnist pid=27967) I0000 00:00:1733410184.043662   28083 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Trial status: 1 RUNNING | 2 PENDING
Current time: 2024-12-05 14:50:01. Total running time: 30s
Logical resource usage: 4.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+---------------------------------------------------------------------------------------------+
| Trial name                status       conv_filters     dropout           lr     batch_size |
+---------------------------------------------------------------------------------------------+
| train_mnist_228ce_00000   RUNNING               167    0.386379   0.0500024              64 |
| train_mnist_228ce_00001   PENDING               193    0.802119   0.0457266             128 |
| train_mnist_228ce_00002   PENDING               164    0.511372   0.00516603            256 |
+---------------------------------------------------------------------------------------------+
Trial status: 1 RUNNING | 2 PENDING
Current time: 2024-12-05 14:50:31. Total running time: 1min 0s
Logical resource usage: 4.0/12 CPUs, 1.0/1 GPUs (0.0/1.

...


Trial train_mnist_228ce_00002 started with configuration:
+--------------------------------------------------+
| Trial train_mnist_228ce_00002 config             |
+--------------------------------------------------+
| batch_size                                   256 |
| conv_filters                                 164 |
| dropout                                  0.51137 |
| lr                                       0.00517 |
+--------------------------------------------------+


(train_mnist pid=30018) 2024-12-05 14:52:11.636987: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_mnist pid=30018) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(train_mnist pid=30018) I0000 00:00:1733410334.200745   30124 service.cc:146] XLA service 0x7b06b8004b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
(train_mnist pid=30018) I0000 00:00:1733410334.200796   30124 service.cc:154]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
(train_mnist pid=30018) I0000 00:00:1733410341.726123   30124 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Trial status: 2 TERMINATED | 1 RUNNING
Current time: 2024-12-05 14:52:31. Total running time: 3min 0s
Logical resource usage: 4.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+--------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                status         conv_filters     dropout           lr     batch_size        acc     iter     total time (s) |
+--------------------------------------------------------------------------------------------------------------------------------------+
| train_mnist_228ce_00002   RUNNING                 164    0.511372   0.00516603            256                                        |
| train_mnist_228ce_00000   TERMINATED              167    0.386379   0.0500024              64   0.103067        1            66.898  |
| train_mnist_228ce_00001   TERMINATED              193    0.802119   0.0457266             128   0.105317        1            79.1913

2024-12-05 14:53:16,003	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/exp' in 0.0041s.



Trial train_mnist_228ce_00002 finished iteration 1 at 2024-12-05 14:53:15. Total running time: 3min 44s
+--------------------------------------------------+
| Trial train_mnist_228ce_00002 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         64.9514 |
| time_total_s                             64.9514 |
| training_iteration                             1 |
| mean_accuracy                            0.99435 |
+--------------------------------------------------+

Trial train_mnist_228ce_00002 completed after 1 iterations at 2024-12-05 14:53:15. Total running time: 3min 44s

Trial status: 3 TERMINATED
Current time: 2024-12-05 14:53:16. Total running time: 3min 44s
Logical resource usage: 4.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
+--------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
best_trial = analysis.get_best_trial("mean_accuracy", "max", "last")
best_accuracy = best_trial.metric_analysis["mean_accuracy"]["last"]
best_trial_config = best_trial.config
print(best_trial_config)
print(best_accuracy)
time_taken = end_time - start_time
print(f"Time taken: {time_taken:.2f} seconds")

{'conv_filters': 164, 'batch_size': 256, 'dropout': 0.5113715690967947, 'lr': 0.005166029382817264}
0.994350016117096
Time taken: 224.82 seconds


### Bayesian


In [ ]:
import time
from ray import train, tune
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# Define LeNet model
class LeNet(nn.Module):
    def __init__(self, num_filters, dropout):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size=5)
        self.conv2 = nn.Conv2d(num_filters, 2 * num_filters, kernel_size=5)
        self.fc1 = nn.Linear(2 * num_filters * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


In [ ]:
!pip show ray
!pip show bayesian-optimization


Name: ray
Version: 2.40.0
Summary: Ray provides a simple, universal API for building distributed applications.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiosignal, click, filelock, frozenlist, jsonschema, msgpack, packaging, protobuf, pyyaml, requests
Required-by: 


In [ ]:
!pip uninstall bayesian-optimization -y


In [ ]:
!pip install bayesian-optimization==1.4.2

In [ ]:
!pip show bayesian-optimization


Name: bayesian-optimization
Version: 1.4.2
Summary: Bayesian Optimization package
Home-page: https://github.com/fmfn/BayesianOptimization
Author: Fernando Nogueira
Author-email: fmfnogueira@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: colorama, numpy, scikit-learn, scipy
Required-by: 


In [ ]:

from ray.air import session

def train_mnist_bayesian(config):
    batch_size_mapping = [64, 128, 256]
    batch_size = batch_size_mapping[int(config["batch_size_idx"])]  # Convert index to actual batch size

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    val_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = LeNet(int(config["num_filters"]), config["dropout"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    start_time = time.time()

    for epoch in range(12):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = correct / total

    total_time = time.time() - start_time
    session.report({"accuracy": accuracy, "time_total_s": total_time})
    print(f"Accuracy: {accuracy:.4f},Time: {total_time:.4f}")


In [ ]:
from ray.tune.search.bayesopt import BayesOptSearch

search_space = {
    "num_filters": tune.uniform(64, 256),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size_idx": tune.uniform(0, 2),  # Index for batch size
    "dropout": tune.uniform(0.0, 1.0)
}

search_alg = BayesOptSearch(metric="accuracy", mode="max")
start_time = time.time()
analysis = tune.run(
    train_mnist_bayesian,
    config=search_space,
    search_alg=search_alg,
    resources_per_trial={"cpu": 1, "gpu": 0.5},
    metric="accuracy",
    mode="max"
)
end_time = time.time()


best_config = analysis.get_best_config(metric="accuracy", mode="max")
batch_size_mapping = [64, 128, 256]
best_config["batch_size"] = batch_size_mapping[int(best_config.pop("batch_size_idx"))]

best_accuracy = analysis.best_result["accuracy"]
# best_time = analysis.best_result["time_total_s"]
time_taken = end_time - start_time





2024-12-05 15:11:46,271	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.


+-----------------------------------------------------------------------------+
| Configuration for experiment     train_mnist_bayesian_2024-12-05_15-11-46   |
+-----------------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                            |
| Scheduler                        FIFOScheduler                              |
| Number of trials                 1                                          |
+-----------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_mnist_bayesian_2024-12-05_15-11-46
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-12-05_15-02-24_292439_278/artifacts/2024-12-05_15-11-46/train_mnist_bayesian_2024-12-05_15-11-46/driver_artifacts`

Trial status: 1 PENDING
Current time: 2024-12-05 15:11:46. Total running time: 0s
Logical resource usage: 1.0/12 CPUs, 0.5/1 GPUs (0.0/1.0 acce

  0%|          | 0.00/9.91M [00:00<?, ?B/s]
  0%|          | 32.8k/9.91M [00:00<01:05, 151kB/s]
  1%|          | 65.5k/9.91M [00:00<01:05, 150kB/s]
  1%|▏         | 131k/9.91M [00:00<00:44, 219kB/s] 
  3%|▎         | 262k/9.91M [00:00<00:26, 369kB/s]
  5%|▌         | 524k/9.91M [00:01<00:14, 668kB/s]
 11%|█         | 1.05M/9.91M [00:01<00:07, 1.26MB/s]
 21%|██        | 2.10M/9.91M [00:01<00:03, 2.41MB/s]
 42%|████▏     | 4.19M/9.91M [00:01<00:01, 4.70MB/s]


(train_mnist_bayesian pid=4628) Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.02MB/s]


...

(train_mnist_bayesian pid=4628) Failed to download (trying next):
(train_mnist_bayesian pid=4628) HTTP Error 403: Forbidden
(train_mnist_bayesian pid=4628) 
(train_mnist_bayesian pid=4628) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz
(train_mnist_bayesian pid=4628) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
(train_mnist_bayesian pid=4628) Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
(train_mnist_bayesian pid=4628) 


100%|██████████| 4.54k/4.54k [00:00<00:00, 14.1MB/s]



Trial status: 1 RUNNING
Current time: 2024-12-05 15:12:16. Total running time: 30s
Logical resource usage: 1.0/12 CPUs, 0.5/1 GPUs (0.0/1.0 accelerator_type:A100)
+-----------------------------------------------------------------------------------------------------+
| Trial name                      status       num_filters          lr     batch_size_idx     dropout |
+-----------------------------------------------------------------------------------------------------+
| train_mnist_bayesian_61e69b58   RUNNING          178.942   0.0734674            0.74908    0.950714 |
+-----------------------------------------------------------------------------------------------------+
Trial status: 1 RUNNING
Current time: 2024-12-05 15:12:46. Total running time: 1min 0s
Logical resource usage: 1.0/12 CPUs, 0.5/1 GPUs (0.0/1.0 accelerator_type:A100)
+-----------------------------------------------------------------------------------------------------+
| Trial name                      status     

2024-12-05 15:15:11,903	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_mnist_bayesian_2024-12-05_15-11-46' in 0.0039s.



Trial train_mnist_bayesian_61e69b58 finished iteration 1 at 2024-12-05 15:15:11. Total running time: 3min 25s
+--------------------------------------------------------+
| Trial train_mnist_bayesian_61e69b58 result             |
+--------------------------------------------------------+
| checkpoint_dir_name                                    |
| time_this_iter_s                               201.275 |
| time_total_s                                   201.275 |
| training_iteration                                   1 |
| accuracy                                        0.0892 |
+--------------------------------------------------------+

Trial train_mnist_bayesian_61e69b58 completed after 1 iterations at 2024-12-05 15:15:11. Total running time: 3min 25s

Trial status: 1 TERMINATED
Current time: 2024-12-05 15:15:11. Total running time: 3min 25s
Logical resource usage: 1.0/12 CPUs, 0.5/1 GPUs (0.0/1.0 accelerator_type:A100)
Current best trial: 61e69b58 with accuracy=0.0892 and params={'num_

In [ ]:
print("Best hyperparameters:", best_config)
print(f"Best validation accuracy: {best_accuracy:.4f}")
print(f"Time taken: {time_taken:.2f} seconds")

Best hyperparameters: {'num_filters': 178.94242896583103, 'lr': 0.07346740023932911, 'dropout': 0.9507143064099162, 'batch_size': 64}
Best validation accuracy: 0.0892
Time taken: 205.67 seconds


## Part 3.

For these three methods, Grid search takes the longest time to find the best model with accuracy is 0.9981. While for the other two, although they are faster, their accuracies are incredible low. For Bayesian, the hyperparameter's range is too big thus it hard to perform well. For Hyperband, it relys on early stopping which may stop too early. Thus Grid Search produced the best model with the highest validation accuracy but at the cost of significantly higher computation time. Both Bayesian Optimization and Hyperband were faster but failed to identify the best hyperparameter configurations if we don't set the stop point, leading to very poor model performance.

# Problem 3

### Setup

- **Learner 1 (L1)** computes gradients:
  - \( g[L_1, 1] \) at \( t = 1 \)
  - \( g[L_1, 2] \) at \( t = 2 \)
  - \( g[L_1, 3] \) at \( t = 3 \)
  - \( g[L_1, 4] \) at \( t = 4 \)

- **Learner 2 (L2)** computes gradients:
  - \( g[L_2, 1] \) at \( t = 2.5 \)
  - \( g[L_2, 2] \) at \( t = 5 \)

---

### Step-by-Step Staleness Calculation

1. **\( g[L_1, 1] \):**
   - Computed at \( t = 1 \).
   - No updates have occurred before this gradient is used.
   - **Staleness = 0**

2. **\( g[L_1, 2] \):**
   - Computed at \( t = 2 \).
   - No updates have occurred before this gradient is used.
   - **Staleness = 0**

3. **\( g[L_2, 1] \):**
   - Computed at \( t = 2.5 \).
   - By this time, \( g[L_1, 1] \) and \( g[L_1, 2] \) have been used to update weights.
   - **Staleness = 2**

4. **\( g[L_1, 3] \):**
   - Computed at \( t = 3 \).
   - By this time, \( g[L_2, 1] \) has been used to update weights.
   - **Staleness = 1**

5. **\( g[L_1, 4] \):**
   - Computed at \( t = 4 \).
   - By this time, \( g[L_1, 3] \) has been used to update weights.
   - **Staleness = 0**

6. **\( g[L_2, 2] \):**
   - Computed at \( t = 5 \).
   - By this time, \( g[L_1, 4] \) and \( g[L_1, 3] \) have been used to update weights.
   - **Staleness = 3**

* $L_1$ : $g[L_1,1]$ at second 1, staleness = 0
* $L_1$ : $g[L_1,2]$ at second 2, staleness = 0
* $L_1$ : $g[L_1,3]$ at second 3, staleness = 1
* $L_1$ : $g[L_1,4]$ at second 4, staleness = 0
* $L_2$ : $g[L_2,1]$ at second 2.5, staleness = 2
* $L_2$ : $g[L_2,2]$ at second 5, staleness = 2


# Problem 4

## Part 1.

In [ ]:
!git clone https://github.com/kuangliu/pytorch-cifar.git
!cd pytorch-cifar


Cloning into 'pytorch-cifar'...
remote: Enumerating objects: 382, done.
remote: Total 382 (delta 0), reused 0 (delta 0), pack-reused 382 (from 1)
Receiving objects: 100% (382/382), 81.31 KiB | 2.39 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
import os
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content


In [ ]:
import sys

# 添加 pytorch-cifar 到 sys.path
sys.path.append('/content/pytorch-cifar')

# 现在尝试再次导入
from models.resnet import ResNet18

In [ ]:
import torch
import torch.nn as nn
from models.resnet import ResNet18
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:

# Assuming you are using the ResNet18 from the cloned repo
model = ResNet18()



In [ ]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170M/170M [00:01<00:00, 94.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ResNet18().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import time

batch_size = 32
times = []

while True:
    try:

        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)


        model.train()
        for data in trainloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()


        start_time = time.time()
        for data in trainloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()
        end_time = time.time()

        epoch_time = end_time - start_time
        times.append(epoch_time)
        print(f"Batch Size: {batch_size}, Time: {epoch_time:.2f} seconds")


        batch_size *= 4

    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(f"Batch size {batch_size} exceeds GPU memory capacity.")
            break
        else:
            raise



for size, t in zip([32 * (4 ** i) for i in range(len(times))], times):
    print(f"Batch Size: {size}, Time: {t:.2f} seconds")


Batch Size: 32, Time: 17.59 seconds
Batch Size: 128, Time: 10.73 seconds
Batch Size: 512, Time: 10.28 seconds
Batch Size: 2048, Time: 10.55 seconds
Batch Size: 8192, Time: 11.66 seconds
Batch size 32768 exceeds GPU memory capacity.
Batch Size: 32, Time: 17.59 seconds
Batch Size: 128, Time: 10.73 seconds
Batch Size: 512, Time: 10.28 seconds
Batch Size: 2048, Time: 10.55 seconds
Batch Size: 8192, Time: 11.66 seconds


## Part 2.

#### GPU - 4

In [ ]:
model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])
model.to(device)

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       

In [ ]:

criterion = nn.CrossEntropyLoss()
import time
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [ ]:
batch_sizes = [32, 128, 512]
results = []

for batch_size in batch_sizes:
    trainloader = DataLoader(trainset, batch_size=batch_size * torch.cuda.device_count(), shuffle=True, num_workers=2)


    for _ in range(1):
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


    start_time = time.time()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    end_time = time.time()

    epoch_time = end_time - start_time
    results.append((batch_size, epoch_time))


for batch_size, timing in results:
    print(f"Batch Size {batch_size}, Time: {timing:.2f} sec")


Batch Size 32, Time: 16.71 sec
Batch Size 128, Time: 9.30 sec
Batch Size 512, Time: 9.26 sec


#### GPU -2

In [ ]:
# GPU 2

model = nn.DataParallel(model, device_ids=[0, 1])
model.to(device)
criterion = nn.CrossEntropyLoss()
import time
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [ ]:
batch_sizes = [32, 128, 512]
results = []

for batch_size in batch_sizes:
    trainloader = DataLoader(trainset, batch_size=batch_size * torch.cuda.device_count(), shuffle=True, num_workers=2)


    for _ in range(1):
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


    start_time = time.time()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    end_time = time.time()

    epoch_time = end_time - start_time
    results.append((batch_size, epoch_time))


for batch_size, timing in results:
    print(f"Batch Size {batch_size}, Time: {timing:.2f} sec")


Batch Size 32, Time: 16.23 sec
Batch Size 128, Time: 9.32 sec
Batch Size 512, Time: 9.48 sec


| GPU Configuration | Batch Size per GPU | Time (sec) | Speedup |
|-------------------|--------------------|------------|---------|
| 1-GPU             | 32                 | 17.59      | 1       |
| 1-GPU             | 128                | 10.73      | 1       |
| 1-GPU             | 512                | 10.28      | 1       |
| 2-GPU             | 32                 | 16.23      | 1.22    |
| 2-GPU             | 128                | 9.32      | 1.15    |
| 2-GPU             | 512                | 9.48      | 1.08    |
| 4-GPU             | 32                 | 16.71      | 1.18    |
| 4-GPU             | 128                | 9.30       | 1.16    |
| 4-GPU             | 512                | 9.26       | 1.08    |


This is Strong-scaling. If we were to use weak scaling instead, where each additional GPU added comes with a proportionate increase in workload to keep the load per GPU consistent, the execution time is ideally meant to stay about the same instead of decreasing. Weak scaling might prove to be better in this scenarios.

## Part 3.

#### Batch Size: 32
- **Single GPU Time**: 17.59 seconds
- **2-GPU Total Time**: 16.23 seconds
- **4-GPU Total Time**: 16.71 seconds
- **Estimated Compute Time** (75% of Single GPU Time): `17.59 * 0.75 = 13.19` seconds
- **2-GPU Communication Time**: `16.23 - 13.19 = 3.04` seconds
- **4-GPU Communication Time**: `16.71 - 13.19 = 3.52` seconds

#### Batch Size: 128
- **Single GPU Time**: 10.76 seconds
- **2-GPU Total Time**: 9.32 seconds
- **4-GPU Total Time**: 9.30 seconds
- **Estimated Compute Time**: `10.73 * 0.75 = 8.05` seconds
- **2-GPU Communication Time**: `9.32 - 8.05 = 1.27` seconds
- **4-GPU Communication Time**: `9.30 - 8.05 = 1.25` seconds

#### Batch Size: 512
- **Single GPU Time**: 10.26 seconds
- **2-GPU Total Time**: 9.48 seconds
- **4-GPU Total Time**: 9.26 seconds
- **Estimated Compute Time**: `10.28 * 0.75 = 7.71` seconds
- **2-GPU Communication Time**: `9.48 - 7.71 = 1.77` seconds
- **4-GPU Communication Time**: `9.26 - 7.71 = 1.55` seconds

### Compiled Table (Complete Data)

| GPU Configuration | Batch Size | Compute(sec) | Comm(sec) |
|-------------------|------------|--------------|-----------|
| 2-GPU             | 32         | 13.19       | 3.04      |
| 4-GPU             | 32         | 13.19        | 3.52     |
| 2-GPU             | 128        | 8.05        | 1.27      |
| 4-GPU             | 128        | 8.05         | 1.25      |
| 2-GPU             | 512        | 7.71         | 1.77      |
| 4-GPU             | 512        | 7.71        | 1.55     |

## Part 4


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

# 定义 ResNet-18 模型
def get_model():
    model = resnet18()
    model.fc = nn.Linear(model.fc.in_features, 10)  # 调整最后一层以适配 CIFAR10 数据集
    return model

# 计算梯度大小（单位：GB）
def calculate_gradient_size(model):
    gradient_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
    gradient_size_bytes = gradient_size * 4
    gradient_size_gb = gradient_size_bytes / (1024**3)
    return gradient_size_gb


def calculate_bandwidth_utilization(num_gpus, batch_sizes, communication_times):
    model = get_model()
    gradient_size_gb = calculate_gradient_size(model)
    results = []

    for batch_size, comm_time in zip(batch_sizes, communication_times):

        bandwidth_utilization = gradient_size_gb / comm_time
        results.append({
            'Batch Size per GPU': batch_size,
            'Communication Time (sec)': comm_time,
            'Bandwidth Utilization (GB/s)': bandwidth_utilization
        })
        print(f"Batch Size per GPU: {batch_size}, Communication Time: {comm_time:.4f}s, Bandwidth Utilization: {bandwidth_utilization:.4f} GB/s")

    return results

if __name__ == "__main__":

    communication_times_2gpu = [3.04, 1.27, 1.77]  # 2 GPUs
    communication_times_4gpu = [3.52, 1.25, 1.55]  # 4 GPUs


    batch_sizes = [32, 128, 512]

    print("\n--- Bandwidth Utilization for 2 GPUs ---")
    results_2gpu = calculate_bandwidth_utilization(2, batch_sizes, communication_times_2gpu)

    print("\n--- Bandwidth Utilization for 4 GPUs ---")
    results_4gpu = calculate_bandwidth_utilization(4, batch_sizes, communication_times_4gpu)



--- Bandwidth Utilization for 2 GPUs ---
Batch Size per GPU: 32, Communication Time: 3.0400s, Bandwidth Utilization: 0.0137 GB/s
Batch Size per GPU: 128, Communication Time: 1.2700s, Bandwidth Utilization: 0.0328 GB/s
Batch Size per GPU: 512, Communication Time: 1.7700s, Bandwidth Utilization: 0.0235 GB/s

--- Bandwidth Utilization for 4 GPUs ---
Batch Size per GPU: 32, Communication Time: 3.5200s, Bandwidth Utilization: 0.0118 GB/s
Batch Size per GPU: 128, Communication Time: 1.2500s, Bandwidth Utilization: 0.0333 GB/s
Batch Size per GPU: 512, Communication Time: 1.5500s, Bandwidth Utilization: 0.0269 GB/s




1.   $ T_{\text{allreduce}} = \left(2 \times (n-1) \times (L+\frac{m}{nb}) \right)$
2.   $ U = \frac{\frac{m}{n}}{b \times (L+\frac{m}{nb})}$
3. Table 3

| GPU Configuration | Batch Size | Bandwidth Utilization (GB/s) |
|-------------------|------------|------------------------------|
| 2-GPU             | 32         | 0.0137                       |
| 2-GPU             | 128        | 0.0328                        |
| 2-GPU             | 512        | 0.0235                      |
| 4-GPU             | 32         | 0.0118                    |
| 4-GPU             | 128        | 0.0333                    |
| 4-GPU             | 512        | 0.0269                      |

